In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_coal_balance(row):
    # Берём ключевые поля из текущей строки
    current_date = row['ВыгрузкаНаСклад']
    warehouse = row['Склад']
    pile = row['Штабель']
    cargo_type = row['Марка']

    # Фильтруем запасы угля на складе до текущей даты
    incoming = supplies[
        (supplies['ВыгрузкаНаСклад'] <= current_date)
        & (supplies['Склад'] == warehouse)
        & (supplies['Штабель'] == pile)
        & (supplies['Марка'] == cargo_type)
        ]['На склад, тн'].sum()

    # Фильтруем объёмы угля, отправленные с судов до текущей даты
    outgoing = supplies[
        (supplies['ПогрузкаНаСудно'] <= current_date)
        & (supplies['Склад'] == warehouse)
        & (supplies['Штабель'] == pile)
        & (supplies['Марка'] == cargo_type)
        ]['На судно, тн'].sum()

    # Баланс равен количеству принятого угля минус отправленному
    balance = incoming - outgoing
    return balance


In [96]:
supplies = pd.read_csv('../data/supplies.csv', parse_dates=['ВыгрузкаНаСклад', 'ПогрузкаНаСудно'])
temperature = pd.read_csv('../data/temperature (1).csv', parse_dates=['Дата акта'])
weather_2015 = pd.read_csv('../data/weather_data_2015.csv', parse_dates=['date'])
weather_2016 = pd.read_csv('../data/weather_data_2016.csv', parse_dates=['date'])
weather_2017 = pd.read_csv('../data/weather_data_2017.csv', parse_dates=['date'])
weather_2018 = pd.read_csv('../data/weather_data_2018.csv', parse_dates=['date'])
weather_2019 = pd.read_csv('../data/weather_data_2019.csv', parse_dates=['date'])
weather_2020 = pd.read_csv('../data/weather_data_2020.csv', parse_dates=['date'])
weather_2021 = pd.read_csv('../data/weather_data_2021.csv', parse_dates=['date'])
weather = pd.concat([weather_2015, weather_2016, weather_2017, weather_2018, weather_2019, weather_2020, weather_2021])
fires = pd.read_csv('../data/fires.csv', parse_dates=['Дата начала', 'Дата оконч.'])

In [101]:
temperature.sort_values(['Дата акта'], inplace=True)
min_date = temperature['Дата акта'].iloc[0]
max_date = temperature['Дата акта'].iloc[-1]

temperature.rename(columns={'Дата акта': 'Дата'}, inplace=True)

supplies.rename(columns={'Наим. ЕТСНГ': 'Марка'}, inplace=True)

fires.rename(columns={'Груз': 'Марка'}, inplace=True)
fires['Дата начала'] = fires['Дата начала'].dt.normalize()

supplies.sort_values(['ВыгрузкаНаСклад'], inplace=True)
fires.sort_values(['Дата начала'], inplace=True)



supplies['Масса угля'] = 0

for i in supplies.index:
    supplies['Масса угля'][i] = calculate_coal_balance(supplies.loc[i])

storage = supplies.copy()
storage.drop_duplicates(subset=['Масса угля'], keep='last', inplace=True)
storage = storage.filter(['ВыгрузкаНаСклад', 'Марка', 'Штабель', 'Склад', 'Масса угля'])
storage.rename(columns={'ВыгрузкаНаСклад': 'Дата'}, inplace=True)

# Генерируем диапазон всех нужных дат
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')

# Уникальные активные комбинации Марка + Штабель + Склад
active_combinations = storage[['Марка', 'Штабель', 'Склад']].drop_duplicates()

# Создаём все даты для активных комбинаций
active_combinations['key'] = 1
all_dates_df = pd.DataFrame({'Дата': all_dates})
all_dates_df['key'] = 1

# Картезианское произведение (все даты × активные штабели)
full_grid = all_dates_df.merge(active_combinations, on='key').drop(columns='key')

# Объединяем с оригинальной таблицей
df_full = full_grid.merge(storage, on=['Дата', 'Марка', 'Штабель', 'Склад'], how='left')

# Заполняем пропуски методом ffill по группам
df_full['Масса угля'] = df_full.groupby(['Марка', 'Штабель', 'Склад'])['Масса угля'].ffill()

# Отсортируем для читаемости
df_result = df_full.sort_values(['Марка', 'Штабель', 'Склад', 'Дата']).reset_index(drop=True)

/tmp/ipykernel_92154/14908574.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  supplies['Масса угля'][i] = calculate_coal_balance(supplies.loc[i])
/tmp/ipykernel_92154/14908574.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '11984.1925' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  supplies['Масса угля'][i] = calculate_coal_balance(supplies.loc[i])


In [102]:
fires = fires[(fires['Дата начала'] >= min_date) & (fires['Дата начала'] <= max_date)]

full_data = df_result.merge(fires, how='left',
                            left_on=['Дата', 'Марка', 'Штабель', 'Склад'],
                            right_on=['Дата начала', 'Марка', 'Штабель', 'Склад'])
full_data

,Дата,Марка,Штабель,Склад,Масса угля,Дата составления,"Вес по акту, тн",Дата начала,Дата оконч.,Нач.форм.штабеля
0,2019-11-23,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN
1,2019-11-24,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN
2,2019-11-25,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN
3,2019-11-26,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN
4,2019-11-27,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...
75824,2020-09-26,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN
75825,2020-09-27,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN
75826,2020-09-28,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN
75827,2020-09-29,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN


In [103]:
full_data['Возгорание'] = (~full_data['Дата начала'].isnull()).astype(int)
full_data

,Дата,Марка,Штабель,Склад,Масса угля,Дата составления,"Вес по акту, тн",Дата начала,Дата оконч.,Нач.форм.штабеля,Возгорание
0,2019-11-23,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN,0
1,2019-11-24,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN,0
2,2019-11-25,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN,0
3,2019-11-26,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN,0
4,2019-11-27,A1,1,3,NaN,NaN,NaN,NaT,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
75824,2020-09-26,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN,0
75825,2020-09-27,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN,0
75826,2020-09-28,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN,0
75827,2020-09-29,L11,34,3,43274.197,NaN,NaN,NaT,NaT,NaN,0


In [104]:
full_data['Возгорание'].value_counts()

Возгорание
0    75524
1      305
Name: count, dtype: int64

In [75]:
full_data = full_data.dropna(subset=['Масса угля']).sort_values(['Дата'])
full_data

,Дата,Марка,Штабель,Склад,Масса угля,Дата составления,"Вес по акту, тн",Дата начала,Дата оконч.,Нач.форм.штабеля,Возгорание
19159,2019-11-23,A1,36,3,238428.6550,NaN,NaN,NaT,NaT,NaN,0
45468,2019-11-23,C3,36,3,25809.4085,NaN,NaN,NaT,NaT,NaN,0
8822,2019-11-23,A1,17,3,32158.8575,NaN,NaN,NaT,NaT,NaN,0
5066,2019-11-23,A1,10,4,22825.6450,NaN,NaN,NaT,NaT,NaN,0
16961,2019-11-24,A1,31,3,263446.7560,NaN,NaN,NaT,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
26374,2020-09-30,A1,50,4,119624.8435,NaN,NaN,NaT,NaT,NaN,0
26687,2020-09-30,A1,51,6,65224.1465,NaN,NaN,NaT,NaT,NaN,0
60804,2020-09-30,E5,26,6,2607.5195,NaN,NaN,NaT,NaT,NaN,0
2560,2020-09-30,A1,5,6,184674.6975,NaN,NaN,NaT,NaT,NaN,0


In [77]:
full_data.drop_duplicates(subset=['Масса угля', 'Дата'], inplace=True)

In [78]:
full_data = full_data[full_data['Масса угля'] != 0]
full_data

,Дата,Марка,Штабель,Склад,Масса угля,Дата составления,"Вес по акту, тн",Дата начала,Дата оконч.,Нач.форм.штабеля,Возгорание
19159,2019-11-23,A1,36,3,238428.6550,NaN,NaN,NaT,NaT,NaN,0
45468,2019-11-23,C3,36,3,25809.4085,NaN,NaN,NaT,NaT,NaN,0
8822,2019-11-23,A1,17,3,32158.8575,NaN,NaN,NaT,NaT,NaN,0
5066,2019-11-23,A1,10,4,22825.6450,NaN,NaN,NaT,NaT,NaN,0
16961,2019-11-24,A1,31,3,263446.7560,NaN,NaN,NaT,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
26374,2020-09-30,A1,50,4,119624.8435,NaN,NaN,NaT,NaT,NaN,0
26687,2020-09-30,A1,51,6,65224.1465,NaN,NaN,NaT,NaT,NaN,0
60804,2020-09-30,E5,26,6,2607.5195,NaN,NaN,NaT,NaT,NaN,0
2560,2020-09-30,A1,5,6,184674.6975,NaN,NaN,NaT,NaT,NaN,0


In [79]:
temperature

,Склад,Штабель,Марка,Максимальная температура,Пикет,Дата,Смена
30,4,10,A1-СУЭК,16.7,NaN,2019-11-23,921.0
22,3,7,A1-СКГ,19.2,NaN,2019-11-23,921.0
23,3,36,A1-МТ-ШУБ,17.7,NaN,2019-11-23,921.0
24,3,17,A1-КРУ,20.5,NaN,2019-11-23,921.0
25,3,31,A1-СА-ШУБ,18.3,NaN,2019-11-23,921.0
...,...,...,...,...,...,...,...
4081,3,33,A1,73.5,3048-3092,2020-09-30,219.0
4080,3,27,A1,34.3,3004-3007,2020-09-30,219.0
4104,6,1,A1,28.4,6035-6053,2020-09-30,921.0
4091,6,1,A1,55.8,6035-6053,2020-09-30,219.0


In [80]:
tmp = full_data.drop(columns=['Дата составления', 'Вес по акту, тн', 'Дата начала', 'Дата оконч.', 'Нач.форм.штабеля'])
tmp = tmp[tmp['Марка'] == 'A1']
tmp = tmp[(tmp['Дата'] >= temperature['Дата'].iloc[0]) & (tmp['Дата'] <= temperature['Дата'].iloc[-1])]
tmp

,Дата,Марка,Штабель,Склад,Масса угля,Возгорание
19159,2019-11-23,A1,36,3,238428.6550,0
8822,2019-11-23,A1,17,3,32158.8575,0
5066,2019-11-23,A1,10,4,22825.6450,0
16961,2019-11-24,A1,31,3,263446.7560,0
24498,2019-11-24,A1,46,4,41344.7905,0
...,...,...,...,...,...,...
25748,2020-09-30,A1,48,3,222639.7860,0
26061,2020-09-30,A1,50,3,22399.0895,0
26374,2020-09-30,A1,50,4,119624.8435,0
26687,2020-09-30,A1,51,6,65224.1465,0


In [81]:
temperature.rename(columns={'Дата акта': 'Дата'}, inplace=True)
temperature['Дата'] = pd.to_datetime(temperature['Дата'])

In [82]:
filtered_temperature = temperature[temperature['Марка'].str.startswith('A1')]
# filtered_temperature['Марка'] = filtered_temperature['Марка'].where(~filtered_temperature['Марка'].str.contains('^A1'), 'A1')
filtered_temperature['Дата'] = pd.to_datetime(filtered_temperature['Дата'], errors='coerce')
filtered_temperature

/tmp/ipykernel_92154/824777246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_temperature['Дата'] = pd.to_datetime(filtered_temperature['Дата'], errors='coerce')


,Склад,Штабель,Марка,Максимальная температура,Пикет,Дата,Смена
30,4,10,A1-СУЭК,16.7,NaN,2019-11-23,921.0
22,3,7,A1-СКГ,19.2,NaN,2019-11-23,921.0
23,3,36,A1-МТ-ШУБ,17.7,NaN,2019-11-23,921.0
24,3,17,A1-КРУ,20.5,NaN,2019-11-23,921.0
25,3,31,A1-СА-ШУБ,18.3,NaN,2019-11-23,921.0
...,...,...,...,...,...,...,...
4081,3,33,A1,73.5,3048-3092,2020-09-30,219.0
4080,3,27,A1,34.3,3004-3007,2020-09-30,219.0
4104,6,1,A1,28.4,6035-6053,2020-09-30,921.0
4091,6,1,A1,55.8,6035-6053,2020-09-30,219.0


In [83]:
tmp = full_data[(full_data['Дата'] >= temperature['Дата'].iloc[0]) & (full_data['Дата'] <= temperature['Дата'].iloc[-1])
                & (full_data['Марка'] == 'A1')]
tmp = tmp.drop(columns=['Марка'])
tmp

,Дата,Штабель,Склад,Масса угля,Дата составления,"Вес по акту, тн",Дата начала,Дата оконч.,Нач.форм.штабеля,Возгорание
19159,2019-11-23,36,3,238428.6550,NaN,NaN,NaT,NaT,NaN,0
8822,2019-11-23,17,3,32158.8575,NaN,NaN,NaT,NaT,NaN,0
5066,2019-11-23,10,4,22825.6450,NaN,NaN,NaT,NaT,NaN,0
16961,2019-11-24,31,3,263446.7560,NaN,NaN,NaT,NaT,NaN,0
24498,2019-11-24,46,4,41344.7905,NaN,NaN,NaT,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...
25748,2020-09-30,48,3,222639.7860,NaN,NaN,NaT,NaT,NaN,0
26061,2020-09-30,50,3,22399.0895,NaN,NaN,NaT,NaT,NaN,0
26374,2020-09-30,50,4,119624.8435,NaN,NaN,NaT,NaT,NaN,0
26687,2020-09-30,51,6,65224.1465,NaN,NaN,NaT,NaT,NaN,0


In [84]:
tmp_temp = filtered_temperature.drop(columns=['Марка'])

In [85]:
tmp = tmp.merge(tmp_temp, how='inner', left_on=['Дата', 'Штабель', 'Склад'], right_on=['Дата', 'Штабель', 'Склад'])
tmp.drop(columns=['Смена'], inplace=True)
tmp.dropna(axis=1, inplace=True)
tmp

,Дата,Штабель,Склад,Масса угля,Возгорание,Максимальная температура
0,2019-11-23,36,3,238428.6550,0,17.7
1,2019-11-23,17,3,32158.8575,0,20.5
2,2019-11-23,10,4,22825.6450,0,16.7
3,2019-11-24,31,3,263446.7560,0,14.8
4,2019-11-24,31,3,263446.7560,0,18.1
...,...,...,...,...,...,...
3885,2020-09-30,45,6,8674.2585,0,30.7
3886,2020-09-30,46,4,129052.8055,0,27.2
3887,2020-09-30,46,4,129052.8055,0,29.4
3888,2020-09-30,5,6,184674.6975,0,65.3


In [86]:
group_cols = ['Дата', 'Склад', 'Штабель', 'Масса угля', 'Возгорание']  # подставь свои реальные имена

# Найдём индексы строк с максимальной температурой в каждой группе
idx = tmp.groupby(group_cols)['Максимальная температура'].idxmax()

# Оставим только строки с максимальной температурой
df_max_temp = tmp.loc[idx].reset_index(drop=True)
df_max_temp

,Дата,Штабель,Склад,Масса угля,Возгорание,Максимальная температура
0,2019-11-23,17,3,32158.8575,0,20.5
1,2019-11-23,36,3,238428.6550,0,17.7
2,2019-11-23,10,4,22825.6450,0,16.7
3,2019-11-24,17,3,32547.1120,0,16.6
4,2019-11-24,31,3,263446.7560,0,18.1
...,...,...,...,...,...,...
2053,2020-09-30,46,4,129052.8055,0,29.4
2054,2020-09-30,1,6,233258.9125,0,55.8
2055,2020-09-30,5,6,184674.6975,0,65.3
2056,2020-09-30,26,6,120530.8670,0,30.4


In [87]:
# Создаём сдвинутую на -3 дня таблицу с нужными колонками
df_shifted = df_max_temp[['Дата', 'Штабель', 'Склад', 'Возгорание']].copy()
df_shifted['Дата'] = df_shifted['Дата'] - pd.Timedelta(days=3)
df_shifted = df_shifted.rename(columns={'Возгорание': 'is_fire'})

# Объединяем по дате, складу и штабелю
df_with_target = df_max_temp.merge(df_shifted, on=['Дата', 'Штабель', 'Склад'], how='left')

# Заполним пропущенные значения (если есть) — например, нулями
# df_with_target['is_fire'] = df_with_target['is_fire'].fillna(0).astype(int)

df_with_target

,Дата,Штабель,Склад,Масса угля,Возгорание,Максимальная температура,is_fire
0,2019-11-23,17,3,32158.8575,0,20.5,0.0
1,2019-11-23,36,3,238428.6550,0,17.7,0.0
2,2019-11-23,10,4,22825.6450,0,16.7,0.0
3,2019-11-24,17,3,32547.1120,0,16.6,0.0
4,2019-11-24,31,3,263446.7560,0,18.1,0.0
...,...,...,...,...,...,...,...
2053,2020-09-30,46,4,129052.8055,0,29.4,NaN
2054,2020-09-30,1,6,233258.9125,0,55.8,NaN
2055,2020-09-30,5,6,184674.6975,0,65.3,NaN
2056,2020-09-30,26,6,120530.8670,0,30.4,NaN


In [88]:
weather.drop(columns=['visibility'], inplace=True)

In [89]:
weather_daily = weather.groupby(weather['date'].dt.date).agg({
    't': ['mean', 'min', 'max'],
    'p': 'mean',
    'humidity': 'mean',
    'precipitation': 'sum',
    'wind_dir': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
    'v_avg': 'mean',
    'v_max': 'max',
    'cloudcover': 'mean',
    'weather_code': lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan,
}).reset_index()
weather_daily['date'] = pd.to_datetime(weather_daily['date'])
# Убираем мультииндекс после агрегации
weather_daily.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in weather_daily.columns]

# Переименуем 'date_' обратно в 'date'
weather_daily = weather_daily.rename(columns={'date_': 'date'})

# Оставляем только нужные колонки
weather_daily = weather_daily[[
    'date',
    't_mean',          # средняя температура
    'humidity_mean',   # средняя влажность
    'precipitation_sum',  # сумма осадков
    'v_avg_mean',      # средняя скорость ветра
    'weather_code_<lambda>'  # тип погоды
]]

# Переименуем weather_code_<lambda> для удобства
weather_daily = weather_daily.rename(columns={
    'weather_code_<lambda>': 'weather_code'
})

In [90]:
weather_daily = weather_daily[(weather_daily['date'] >= temperature['Дата'].iloc[0]) & (weather_daily['date'] <= temperature['Дата'].iloc[-1])]
weather_daily

,date,t_mean,humidity_mean,precipitation_sum,v_avg_mean,weather_code
1786,2019-11-23,2.254167,75.916667,0.0,38.029167,0
1787,2019-11-24,3.379167,77.833333,0.0,29.037500,3
1788,2019-11-25,5.375000,83.250000,0.1,23.812500,3
1789,2019-11-26,7.733333,79.625000,1.3,27.516667,3
1790,2019-11-27,11.083333,89.333333,3.7,20.954167,51
...,...,...,...,...,...,...
2093,2020-09-26,21.545833,82.000000,0.0,13.000000,0
2094,2020-09-27,22.458333,91.083333,0.1,19.654167,1
2095,2020-09-28,21.470833,76.708333,0.0,18.729167,0
2096,2020-09-29,22.441667,88.083333,0.1,18.458333,3


In [91]:
final_data = df_with_target.merge(weather_daily, how='inner', left_on=['Дата'], right_on=['date'])
final_data

,Дата,Штабель,Склад,Масса угля,Возгорание,Максимальная температура,is_fire,date,t_mean,humidity_mean,precipitation_sum,v_avg_mean,weather_code
0,2019-11-23,17,3,32158.8575,0,20.5,0.0,2019-11-23,2.254167,75.916667,0.0,38.029167,0
1,2019-11-23,36,3,238428.6550,0,17.7,0.0,2019-11-23,2.254167,75.916667,0.0,38.029167,0
2,2019-11-23,10,4,22825.6450,0,16.7,0.0,2019-11-23,2.254167,75.916667,0.0,38.029167,0
3,2019-11-24,17,3,32547.1120,0,16.6,0.0,2019-11-24,3.379167,77.833333,0.0,29.037500,3
4,2019-11-24,31,3,263446.7560,0,18.1,0.0,2019-11-24,3.379167,77.833333,0.0,29.037500,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053,2020-09-30,46,4,129052.8055,0,29.4,NaN,2020-09-30,20.650000,81.458333,10.2,17.550000,0
2054,2020-09-30,1,6,233258.9125,0,55.8,NaN,2020-09-30,20.650000,81.458333,10.2,17.550000,0
2055,2020-09-30,5,6,184674.6975,0,65.3,NaN,2020-09-30,20.650000,81.458333,10.2,17.550000,0
2056,2020-09-30,26,6,120530.8670,0,30.4,NaN,2020-09-30,20.650000,81.458333,10.2,17.550000,0


In [98]:
final_data.dropna(inplace=True)

In [99]:
final_data['Возгорание'].value_counts()

Возгорание
0    1532
1     207
Name: count, dtype: int64

In [100]:
final_data[final_data['is_fire'] == 1]

,Дата,Штабель,Склад,Масса угля,Возгорание,Максимальная температура,is_fire,date,t_mean,humidity_mean,precipitation_sum,v_avg_mean,weather_code
433,2020-05-11,6,4,198518.0525,0,24.7,1.0,2020-05-11,13.725000,84.208333,0.0,22.987500,0
625,2020-05-29,1,6,554091.1405,0,28.1,1.0,2020-05-29,15.566667,84.666667,10.8,18.950000,3
636,2020-05-30,1,6,554091.1405,0,30.5,1.0,2020-05-30,15.604167,82.083333,0.6,37.037500,0
682,2020-06-03,1,6,554091.1405,0,32.1,1.0,2020-06-03,16.058333,84.000000,0.0,19.191667,0
692,2020-06-04,1,6,554091.1405,0,31.6,1.0,2020-06-04,15.508333,77.000000,0.5,16.866667,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970,2020-09-24,5,6,184674.6975,1,93.3,1.0,2020-09-24,20.850000,87.708333,0.0,9.120833,0
1976,2020-09-25,33,3,350350.6115,1,254.0,1.0,2020-09-25,21.629167,77.958333,0.0,8.458333,0
1983,2020-09-25,1,6,233258.9125,1,80.8,1.0,2020-09-25,21.629167,77.958333,0.0,8.458333,0
1991,2020-09-26,33,3,350350.6115,0,38.3,1.0,2020-09-26,21.545833,82.000000,0.0,13.000000,0


In [110]:
from preprocessing_1 import *
df = prepare_and_merge_data()
df

KeyboardInterrupt: 